<a href="https://colab.research.google.com/github/Andy14py/my_projects/blob/main/DataScience_Project_Extraccion_Mundial_Qatar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center> **PREDECIR CAMPEON DEL MUNDIAL QATAR** <center>

# FASE 1: Extraccion de data
En vista de que no disponemos de la información requerida, vamos a proceder a extraer datos de una fuente externa utilizando técnicas de web scraping

## Extrayendo data con pandas

In [ ]:
#Empezamos por importar las librerias que nos permiten leer la data en este caso sera proveniente de una pagina web
import pandas as pd 
from string import ascii_uppercase as alfabeto
import pickle

In [ ]:
# recopilamos la data con el metodo pd.read_ht
todas_tablas=pd.read_html('https://en.wikipedia.org/wiki/2022_FIFA_World_Cup')
#La logica nos indica que las tablas estan enumeradas del 10 al 59 en intervalos de 7 desde el grupo A al grupo H
# A->H
# 10 -> 7*7+ 10=59
todas_tablas[10]
todas_tablas[17]
todas_tablas[24]


dict_tablas={}
for letra, i in zip(alfabeto,range(10,66,7)):
    df=todas_tablas[i]
    df.rename(columns={df.columns[1]:"Team"}, inplace=True)
    df.pop("Qualification")
    dict_tablas[f"Grupo {letra}"]=df

with open("dict_table","wb") as output:
    pickle.dump(dict_tablas,output)

dict_tablas['Grupo A']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Netherlands,3,2,1,0,5,1,+4,7
1,2,Senegal,3,2,0,1,5,4,+1,6
2,3,Ecuador,3,1,1,1,4,3,+1,4
3,4,Qatar (H),3,0,0,3,1,7,−6,0


## Extrayendo data con BeautifulSoup

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests



years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974,
         1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014,
         2018]

def get_matches(year):
    web=f"https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup"
    response=requests.get(web)
    content=response.text
    soup=BeautifulSoup(content,"lxml")
#identificamos el nombre de tag y luego el nombre de clase: find_all devuelve una lista
    matches=soup.find_all("div",class_="footballbox")
# Creamos listas vacias que seran llenadas con el metodo append luego de obtener el texto de la web
    home=[]
    score=[]
    away=[]
    for match in matches:
        home.append(match.find("th",class_="fhome").get_text())
        score.append(match.find("th",class_="fscore").get_text())
        away.append(match.find("th",class_="faway").get_text())
#Creamos un diccionario para ordenar la informacion luego se crea un df con pandas
    dict_football={"home":home,"score":score,"away":away} 
    df_football=pd.DataFrame(dict_football)
# Creamos una nueva columna para representar el año del mundial al que se refiere
    df_football["year"]= year
    return df_football
# Creamos lista de comprension con bucle para obtener el nuevo dataframe pero de todos los años en lista years
fifa=[get_matches(year) for year in years]
#Creamos el dataframe con concat
df_fifa=pd.concat(fifa, ignore_index=True)
#exportamos nuestros dos DataFrames df_fifa y df_fixture
df_fifa.to_csv("fifa_worldcup_historical_data.csv", index=False)

df_fixture=get_matches("2022")
df_fixture.to_csv("fifa_worldcup_fixture.csv", index=False)


## Extrayendo data con Selenium

In [ ]:
#Este archivo debe ser ejecutado en Jupiternotebook y descargar el chromedriver.exe que corresponde a su version de Chrome 
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import pandas as pd 
import time

years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974,
         1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014,
         2018]

# el path debe describir la ruta donde esta el archivo chromedriver descargado
path= "C:/Users/Andy/Downloads"
service =Service(executable_path=path)
driver = webdriver.Chrome(service=service)

def obtener_data_faltante(year):
    # usamos la tecnica f string para hacer un link dinamico
    web=f"https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup"
    driver.get(web)

#creacion del "xpath" esto es primero inspeccionando el elemento, obtener el tag el nombre de clases
# y el valor de la misma luego con ".." se encuentra el nodo padre
    matches=driver.find_elements(by="xpath",value="//tr[@itemprop='name']")
#Creamos listas vacias para llenarlas con la data recopilada
    home2=[]
    score2=[]
    away2=[]

    for match in matches:
#el punto(.) representa la importacion de la ruta anterior definida en matches(//tr[@itemprop='name'])
# por ello no se necesita poner toda la ruta completa, ademas los numeros representa el index 
#de la posicion que deseamos encontrar
        home2.append(match.find_element(by='xpath', value='./th[1]').text)
        score2.append(match.find_element(by='xpath', value='./th[2]').text)
        away2.append(match.find_element(by='xpath', value='./th[3]').text)

#Creamos un diccionario para rellenarlo de la informacion descargada anteriormente y luego creamos un dataframe con pandas
# con el metodo pd.DataFrame()
    dict_football2={"home":home2,"score":score2,"away":away2}
    df_football2=pd.DataFrame(dict_football2)
    df_football2["year"]=year
# Damos un delay de 2 segundos intervalo de tiempo que tomara para cerrar y abrir otra pagina
    time.sleep(2)
    return df_football2

# Creamos una lista de comprension que permite por medio del bucle recolectar toda la data necesaria
#definida por los años en la lista years
fifa2=[obtener_data_faltante(year) for year in years]
# Este metodo nos permite cerrar el navegador y terminar la operacion
driver.quit()
# Concatenamos la lista ignorando el index que contengan y redefiniendo uno nuevo
df_fifa2=pd.concat(fifa2,ignore_index=True)
# exportamos la data como archivo csv sin exportar el index
df_fifa2.to_csv("fifa_worldcup_missing_data.csv", index=False)

